In [ ]:
import numpy as np
import pandas as pd
import csv
from sklearn.ensemble import RandomForestClassifier
from flask import Flask, jsonify, request
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.exceptions import NotFittedError

%matplotlib inline

# Import the iris file
df = pd.read_csv("data/iris.csv")
df_input = df.iloc[:, :-1].values
df_output = df.iloc[:,-1:].values.flatten()
df.head()

In [ ]:
app = Flask(__name__)

classifier = RandomForestClassifier()

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average='macro'),
           'recall' : make_scorer(recall_score, average='macro'), 
           'f1_score' : make_scorer(f1_score, average='macro')}

@app.route('/train')
def train(): # http://0.0.0.0:5000/train?n_estimators=10&max_depth=2
    global classifier
    
    #Get the parameters from URL
    n_estimators = int(request.args.get('n_estimators'))
    max_depth = int(request.args.get('max_depth'))
    
    #Set params for classifier
    classifier = classifier.set_params(n_estimators=n_estimators, max_depth=max_depth)
    
    #Train the model
    classifier.fit(df_input, df_output)
    
    #Using cross validation
    results = cross_validate(estimator=classifier,
                         X=df_input,
                         y=df_output,
                         cv=5,
                         scoring=scoring, 
                         return_train_score=True)
    
    #Take the mean for the results of the crossvalidation
    accuracy = results['test_accuracy'].mean()
    f1 = results['test_f1_score'].mean()
    precision = results['test_precision'].mean()
    recall = results['test_recall'].mean()
    
    return jsonify({'f1': f1,
                    'precision': precision,
                    'recall': recall,
                    'accuracy': accuracy})

@app.route('/predict')
def predict(): # http://0.0.0.0:5000/predict?sepal_length=10&sepal_width=45&petal_length=9&petal_width=12
    #Get the parameters from URL
    sepal_length = request.args.get('sepal_length')
    sepal_width = request.args.get('sepal_width')
    petal_length = request.args.get('petal_length')
    petal_width = request.args.get('petal_width')
    
    
    #Predict new flower
    data = {'sepal_length': [sepal_length], 'sepal_width': [sepal_width], 'petal_length': [petal_length], 'petal_width': [petal_width]}
    df = pd.DataFrame(data=data)
    
    #If you don't call route '/train' first, the model returns the mentioned string below
    try:
        y_pred = classifier.predict(df)
    except NotFittedError as e:
        return "You have to train model first. Therefore, call '/train'!"
    
    return jsonify({'iris flower class': y_pred[0]})

app.run(debug=True, use_reloader=False, host='0.0.0.0')

In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn import metrics

%matplotlib inline

# Import two files as dataframe
df_input = pd.read_csv("data/mnist_data.csv", header=None).iloc[:,1:]
df_output = pd.read_csv("data/mnist_target.csv", header=None).iloc[:,1:]

print("Shape:", df_input.shape, "   Shape output:", df_output.shape)

#Notice: test_size=0.75 is not a good value. test_size should be around 0.3
X_train, X_test, y_train, y_test = train_test_split(df_input, df_output, random_state=1, test_size=0.75)
X_train.shape

#Build the decision tree and train it with the training set
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

#Make a prediction with the trained model on the test set
y_pred = dt.predict(X_test)

#Calculate the accuracy score 
score = dt.score(X_test, y_test)
#print the confusion matrix and compare predicted data with actual data
cm = metrics.confusion_matrix(y_test, y_pred)

#Plot the results in a confusion metrics with the accuracy score
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size = 15);